# 2.3 - Macrobond Data API for Python - Filter with a Keyword

*Performing coverage checks based on a keyword*

This notebook aims to provide examples of how to use Macrobond Data API for Python as well as insights on the key attributes used to display the output in an understandable format.

We will focus here on using the Search method based on a **keyword** filter. This helps you build a list of time series potentially relating to a common theme.

You can find a full description of all methods and parameters used in the examples in the [documentation of the API](https://macrobond.github.io/macrobond-data-api/common/api.html).

*The examples uses the common functions of Macrobond API. Full error handling is omitted for brevity*

***

## Importing packages

In [1]:
import macrobond_data_api as mda
import matplotlib.pyplot as plt

mbnd_colors = [
    "#1B365D",
    "#4964A2",
    "#256BA2",
    "#8E51A8",
    "#58A291",
    "#CD545B",
    "#004F59",
    "#F68D2E",
    "#008578",
    "#934054",
    "#25282A",
    "#333F48",
    "#676d72",
    "#C7C9C7",
]

***

## Get the data - Keyword: Government Securities Auctions

Note that we are using here the Search endpoint with filters on keyword `text=Government Securities Auctions` and `Frequency=daily` time series in this example.

In [2]:
data_frame = mda.entity_search(
    text="Government Securities Auctions",
    entity_types="TimeSeries",
    must_have_values={"Frequency": "daily"},
    include_discontinued=False,
).to_pd_data_frame()[
    [
        "Name",
        "FullDescription",
        "Description",
        "Region",
        "Category",
        "Frequency",
        "Source",
        "Class",
        "PriceType",
    ]
]

In [3]:
data_frame

### Let's make sense of the attribute PriceType

You can get more information about an attribute value from the database. This is explained in the notebook [1.1 - Metadata Navigation](./1.1%20-%20Macrobond%20Data%20API%20-%20Metadata%20Navigation.ipynb).

In [4]:
mda.metadata_get_attribute_information("PriceType")

As the PriceType uses values from list and can list values, let's have a look at its values to be used later as a group.

In [5]:
data_frame_PriceType = mda.metadata_list_values("PriceType").to_pd_data_frame()[["value", "description", "comment"]]
data_frame_PriceType

Now that we get a sense of potential groupings we can achieve thanks to the metadata, let's manipulate the ones that will be used.

In [6]:
data_frame["RegionString"] = data_frame["Region"].map(lambda x: x[0])


data_frame["PriceTypeString"] = data_frame["PriceType"].map(lambda x: x if type(x) != list else x[0])

# Mapping the PriceType to its descriptions
mapping = dict(data_frame_PriceType[["value", "description"]].values)
data_frame["PriceTypeD"] = data_frame["PriceTypeString"].map(mapping)
data_frame

#### Displaying the new DataFrame
Let's see how our transformations have been applied by isolating on a few columns: `df.iloc[rows,[columns]]`.

In [11]:
df_final = data_frame[["Name", "FullDescription", "RegionString", "PriceTypeString", "PriceTypeD"]]
df_final.head(10)

### Group the results by PriceType

In [12]:
df_group1 = df_final.groupby(["PriceTypeD"])["Name"].count().reset_index(name="Count")
df_group1.sort_values(by="Count", ascending=False)

Let's isolate the top 10 results of our dataframe to keep the top price types.

In [14]:
df_group1_top = df_group1.nlargest(10, "Count")
df_group1_top

### Group the results by Region

In [16]:
data_frame_region = mda.metadata_list_values("Region").to_pd_data_frame()[["value", "description"]]

# Mapping the Region value to its description
mapping = dict(data_frame_region[["value", "description"]].values)
df_final["RegionD"] = data_frame.RegionString.map(mapping)

# Applying the grouping
df_group2 = df_final.groupby(["RegionD", "PriceTypeD"])["Name"].count().reset_index(name="Count")
df_group2.sort_values(by="Count", ascending=False)
df_group2

In [17]:
heatmap = df_group2.loc[
    (df_group2["RegionD"] == "Poland")
    & (
        (df_group2["PriceTypeD"] == "Yield")
        | (df_group2["PriceTypeD"] == "Offered")
        | (df_group2["PriceTypeD"] == "Price")
        | (df_group2["PriceTypeD"] == "Sales")
    )
    | (df_group2["RegionD"] == "United States")
    & (
        (df_group2["PriceTypeD"] == "Yield")
        | (df_group2["PriceTypeD"] == "Offered")
        | (df_group2["PriceTypeD"] == "Price")
        | (df_group2["PriceTypeD"] == "Sales")
    )
    | (df_group2["RegionD"] == "Sweden")
    & (
        (df_group2["PriceTypeD"] == "Yield")
        | (df_group2["PriceTypeD"] == "Offered")
        | (df_group2["PriceTypeD"] == "Price")
        | (df_group2["PriceTypeD"] == "Sales")
    )
]
heatmap

### Plot the Government Securities Auctions Top by type and region

In [20]:
plt.figure(figsize=(8, 5))

colours = mbnd_colors
plt.bar(df_group1_top["PriceTypeD"], df_group1_top["Count"], color=colours)
plt.title("Pricing Type of Government Securities Auctions", fontsize=14)
plt.xlabel("PriceType", fontsize=14)
plt.ylabel("Count", fontsize=14)
plt.xticks(rotation=45)
plt.grid(False)
plt.autoscale()
plt.show()

Let's visualise the Price Type for Poland and Sweden only.

In [21]:
df_final_filtered = df_final.loc[(df_final["RegionD"] == "Poland") | (df_final["RegionD"] == "Sweden")]

In [23]:
colours = mbnd_colors
df_final_filtered.groupby(["RegionD", "PriceTypeD"]).size().unstack().plot(
    kind="bar", stacked=True, color=colours, figsize=(10, 5)
)

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0, ncol=2)
plt.show()